In [1]:
from pathlib import Path
import time
from typing import Tuple

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

I1119 073938.907 _utils_internal.py:281] NCCL_DEBUG env var is set to None
I1119 073938.909 _utils_internal.py:290] NCCL_DEBUG is WARN from /etc/nccl.conf


In [2]:
FEAT_DIM = 784
NUM_EPOCHS = 10
NUM_LAYERS = 4
BATCH_SIZE = 64
LEARNING_RATE = 0.1

In [3]:
class MLP(nn.Module):
    def __init__(self, feat_dim: int, num_layers: int) -> None:
        super(MLP, self).__init__()
        layers = []
        for i in range(num_layers):
            if i < num_layers - 1:
                layers.append(nn.Linear(feat_dim, feat_dim))
                layers.append(nn.ReLU())
            else:
                layers.append(nn.Linear(feat_dim, 10))
        self.model = nn.Sequential(*layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x.view(-1, FEAT_DIM))

In [4]:
def get_dataloaders(data_dir: Path) -> Tuple[DataLoader, DataLoader]:
    transform = transform = transforms.ToTensor() # Normalise between 0 and 1
    train_data = datasets.MNIST(
        data_dir, train=True, download=True, transform=transform
    )
    val_data = datasets.MNIST(
        data_dir, train=False, download=True, transform=transform
    )
    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False)
    return train_loader, val_loader

In [5]:
def validate(
    val_loader: DataLoader,
    model: nn.Module,
    criterion: nn.Module,
) -> float:
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images.cuda())
            loss = criterion(outputs, labels.cuda())
            val_loss += loss.item()

    val_loss /= len(val_loader)
    return val_loss

In [6]:
def train_loop(
    train_loader: DataLoader,
    val_loader: DataLoader,
    model: nn.Module,
    criterion: nn.Module,
    optimizer: optim.Optimizer,
    log_filepath: Path,
    verbose: bool,
) -> None:
    with log_filepath.open("w") as f:
        for epoch in range(NUM_EPOCHS):
            start = time.time()
            model.train()
            for images, labels in train_loader:
                optimizer.zero_grad()
                outputs = model(images.cuda())
                loss = criterion(outputs, labels.cuda())
                f.write(f"{loss.item()}\n")
                loss.backward()
                optimizer.step()

            val_loss = validate(val_loader, model, criterion)
            print(
                f"Epoch {epoch + 1}/{NUM_EPOCHS}, "
                f"Val Loss: {val_loss:.4f}, "
                f"Duration: {time.time() - start:.2f}s"
            )

In [7]:
def main(data_dir: Path, log_file: Path, verbose: bool) -> None:
    train_loader, val_loader = get_dataloaders(data_dir)
    model = MLP(FEAT_DIM, NUM_LAYERS).cuda()
    model = torch.compile(model)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)

    train_loop(train_loader, val_loader, model, criterion, optimizer, log_file, verbose)
    print("Training complete.")

In [8]:
data_dir = ""  # Fill me out
log_file = Path("")  # Fill me out
verbose = False  # Whether to save train losses to log file
main(data_dir, log_file, verbose)

Epoch 1/10, Val Loss: 0.1813, Duration: 5.52s
Epoch 2/10, Val Loss: 0.1257, Duration: 3.66s
Epoch 3/10, Val Loss: 0.0899, Duration: 3.65s
Epoch 4/10, Val Loss: 0.0735, Duration: 3.65s
Epoch 5/10, Val Loss: 0.0700, Duration: 3.66s
Epoch 6/10, Val Loss: 0.0604, Duration: 3.59s
Epoch 7/10, Val Loss: 0.0620, Duration: 3.57s
Epoch 8/10, Val Loss: 0.0686, Duration: 3.59s
Epoch 9/10, Val Loss: 0.0692, Duration: 3.62s
Epoch 10/10, Val Loss: 0.0664, Duration: 3.63s
Training complete.
